In [1]:
%reset -f
import numpy as np
#import random as rm
import timeit
import itertools
%load_ext line_profiler
from SequenceSim import experiments_markov_binary

In [2]:
# does it work for a second order?
# simulate some probabilities

test_n = 50
test_order = 1


nucleotides = 'ATGC'

file = ["data/MA0024.1.pfm", "data/MA0024.1.pfm"]

# List with values that are transitioned to 
test_transition_states = ["A", "T", "G", "C"]

order_nucleotides = [''.join(p) for p in itertools.product(nucleotides, repeat = test_order)]

prob = np.repeat(1/len(order_nucleotides), len(order_nucleotides))
test_init_freq = dict(zip(order_nucleotides, prob))

#prob = [list(np.repeat(1/len(nucleotides), len(nucleotides))) for i in range(len(order_nucleotides))]
#test_transition_dic = dict(zip(order_nucleotides, prob))


prob = [list(np.random.uniform(low=1, high=1, size= len(test_transition_states))) for i in range(len(order_nucleotides))]
prob = [i/sum(i) for i in prob]
cases_transition_dic = dict(zip(order_nucleotides, prob))


prob = [list(np.random.uniform(low=1, high=1, size= len(test_transition_states))) for i in range(len(order_nucleotides))]
prob = [i/sum(i) for i in prob]
controls_transition_dic = dict(zip(order_nucleotides, prob))


cases_seq_parameters = {'init_freq': test_init_freq,\
                       'transition_dic': cases_transition_dic,\
                       'transition_states': test_transition_states,\
                       'n': test_n,\
                       'order': test_order}

controls_seq_parameters = {'init_freq': test_init_freq,\
                       'transition_dic': controls_transition_dic,\
                       'transition_states': test_transition_states,\
                       'n': test_n,\
                       'order': test_order}


In [7]:
#timeit.timeit('markov_simulations(test_seq_parameters, 10000)', globals = globals(), number = 1)
#size = 100
#X = markov_simulations(test_seq_parameters, size)
#y = np.random.choice(a = [1, 0], p = [0.5, 0.5], size = size).reshape(size, 1)

X, y = experiments_markov_binary(cases_seq_parameters, controls_seq_parameters, 5000, 5000,\
                                 cases_motif_file = file, cases_index = [[10, 15], [25, 30]], cases_presence = [1, .25], \
                                 controls_motif_file = file, controls_index = [[10, 15], [25, 30]], controls_presence = [0, 0.75])

## Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

#print(np.array(X))

In [8]:
#import keras
#from keras.models import Sequential
#from keras.layers import Dense, Flatten
#from keras.layers import Conv1D, MaxPooling1D

import keras.layers as kl
from keras.models import Sequential

In [9]:

# construct the model ----------------------------------------------------------
kl.model = Sequential()
kl.model.add(kl.Conv1D(20, kernel_size=(4), activation='relu', input_shape=(test_n, 4), padding='same'))
#kl.model.add(kl.Dense(100, input_shape=(test_n, 4), activation='relu'))
kl.model.add(kl.MaxPooling1D(pool_size=10))
kl.model.add(kl.Flatten())
kl.model.add(kl.Dense(25, activation = 'relu'))
kl.model.add(kl.Dense(5, activation = 'relu'))
#kl.model.add(kl.Dense(10, activation = 'relu'))
kl.model.add(kl.Dense(1, activation='sigmoid'))
# Compile model
kl.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# compile
kl.model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# print model summary
kl.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 50, 20)            340       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 5, 20)             0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 25)                2525      
                                                                 
 dense_4 (Dense)             (None, 5)                 130       
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                      

In [10]:

# Train ------------------------------------------------------------------------
kl.model.fit(X_train, y_train,
          verbose=1,
             epochs = 10, 
          validation_data=(X_test, y_test))

Epoch 1/10
219/219 [==============================] - 1s 3ms/step - loss: 0.4178 - accuracy: 0.8537 - val_loss: 0.1360 - val_accuracy: 0.9707
Epoch 2/10
219/219 [==============================] - 0s 2ms/step - loss: 0.0725 - accuracy: 0.9817 - val_loss: 0.0394 - val_accuracy: 0.9910
Epoch 3/10
219/219 [==============================] - 0s 2ms/step - loss: 0.0338 - accuracy: 0.9901 - val_loss: 0.0241 - val_accuracy: 0.9947
Epoch 4/10
219/219 [==============================] - 0s 2ms/step - loss: 0.0226 - accuracy: 0.9947 - val_loss: 0.0239 - val_accuracy: 0.9957
Epoch 5/10
219/219 [==============================] - 0s 2ms/step - loss: 0.0177 - accuracy: 0.9959 - val_loss: 0.0153 - val_accuracy: 0.9960
Epoch 6/10
219/219 [==============================] - 0s 2ms/step - loss: 0.0137 - accuracy: 0.9960 - val_loss: 0.0135 - val_accuracy: 0.9967
Epoch 7/10
219/219 [==============================] - 0s 2ms/step - loss: 0.0118 - accuracy: 0.9973 - val_loss: 0.0158 - val_accuracy: 0.9967
Epoch 

In [ ]:
# next steps 
# motif lists
# files, indexes, gaps, orientation (absolute & relative), percentages?